다음 링크의 코드를 참고하였습니다. 제 코드는 대댓글을 수집하지 않습니다. 대댓글 수집 코드는 2번째 링크를 참고해주세요.
- https://velog.io/@kjh1337/Selenium%EA%B3%BC-BeatifulSoup%EB%A5%BC-%EC%82%AC%EC%9A%A9%ED%95%9C-%EC%9C%A0%ED%8A%9C%EB%B8%8C-%EB%8C%93%EA%B8%80-%ED%81%AC%EB%A1%A4%EB%A7%81
- https://project-notwork.tistory.com/40

### Import Library

In [1]:
from selenium import webdriver
import time
from openpyxl import Workbook
import pandas as pd
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

### 엑셀 오픈 & 크롤링 할 유튜브 페이지 불러오기

In [2]:
wb = Workbook(write_only=True)
ws = wb.create_sheet()

driver = webdriver.Chrome("chromedriver.exe")
driver.get("https://www.youtube.com/watch?v=AzVjgIvihXU") # 크롤링 하고자 하는 유튜브 링크를 넣는 부분
driver.implicitly_wait(3)

time.sleep(1.5)

driver.execute_script("window.scrollTo(0, 800)")
time.sleep(3)
driver.execute_script("window.scrollTo(0, window.scrollY-300)") # 위로 한 번 올려줘야 댓글이 업데이트 됨
time.sleep(3)

### 유튜브 프리미엄 팝업 닫기

In [3]:
try:
    driver.find_element_by_css_selector("#dismiss-button > a").click()
except:
    pass

### 웹 페이지 끝까지 스크롤하기
- 모든 댓글을 수집하기 위해서 웹페이지 끝까지 스크롤링을 하고 크롤링해야합니다. 이 코드는 웹 페이지 끝까지 스크롤링하는 코드입니다.

In [4]:
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(1.5)

    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

time.sleep(1.5)

### 댓글 가져오기

In [5]:
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')

id_list = soup.select("div#header-author > h3 > #author-text > span")
comment_list = soup.select("yt-formatted-string#content-text")

id_final = []
comment_final = []

for i in range(len(comment_list)):
    temp_id = id_list[i].text
    temp_id = temp_id.replace('\n', '')
    temp_id = temp_id.replace('\t', '')
    temp_id = temp_id.replace('    ', '')
    id_final.append(temp_id) # 댓글 작성자

    temp_comment = comment_list[i].text
    temp_comment = temp_comment.replace('\n', '')
    temp_comment = temp_comment.replace('\t', '')
    temp_comment = temp_comment.replace('    ', '')
    comment_final.append(temp_comment) # 댓글 내용

### 댓글 저장하기

In [6]:
pd_data = {"id" : id_final , "comment" : comment_final}
youtube_pd = pd.DataFrame(pd_data)

youtube_pd.to_excel('battleground_comment.xlsx')